<a href="https://colab.research.google.com/github/pedroconcejero/deep_learning_CNN/blob/main/%5BPrivate%5D%5BU_tad%5D_VGG16_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'private-u-tad-dogs-vs-cats-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F85781%2F9710566%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241014%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241014T180044Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4b07b60cb328a20d2a6e8af03ffecbe96d02dc05f209859de9ef80035300e748027c467aedecc979df87f12f4f14a6fadcb68a7436b66144cddb921869091481a7a48c7ae36ee0d2bf04ac09919fd8557397a713fa35c291b6c307424a2612bb03787b1705d182856834df53dcb4c3822253f10638bcbe993e1d4b0a364a18d0cd68f43bc4a67551ffaf92fed760b2872c422985d30376e667ccca0782e342af4e0396a7389c1e18aaacdc65ef7f5843ba2d31db5af2ab3aa4cbe7b784044b3639dbbeccb93eb2f9797e6db4fe2eae30921a215c5c78c3b403aca310cb5e9eb6ff088bd0930a33c1312189aa4897bab19fcd22f2748650a5940e35450dbfb605'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import data as tf_data
import keras

seed = 1
keras.utils.set_random_seed(seed)

### Read in the training data

In [ ]:
image_size = (224, 224)

# when working with 20_000 files for training this
# will lead to exactly 160 mini-batches per epoch
batch_size = 125

# https://keras.io/api/data_loading/image/#imagedatasetfromdirectory-function

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    #"PetImages",
    "/kaggle/input/private-u-tad-dogs-vs-cats-2024/train/train",
    validation_split=0.2,
    subset="both",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="categorical",
)

# VGG16 pre trained model

### Importar Keras

Incluímos el modelo VGG16 y vemos sus capas.

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

### Capas

Usamos la misma capa de entrada, quitamos la última capa ya que es la de Image net, creamos nuestra propia capa densa con 2 neuronas, que servirán para: Gato, Perro.

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense
# we create a new input with the requiered parameters, vgg uses a different one that does not work with this input size.
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(2, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)

### Congelar Capas

Como el modelo ya está pre entrenado, podemos congelar todos los pesos y biases de las capas excepto la última, la cual servirá para tomar la decisción de perro o gato. Cabe aclarar que se menciona que en casos con más imagenes de entrenamiento es posible descongelarlas.

In [ ]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True


model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

In [ ]:
epochs = 10

history = model_new.fit(train_ds,
                    validation_data = val_ds,
                    epochs = epochs)



### Compile and train (fit)

In [ ]:
logs = pd.DataFrame(history.history)

plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.plot(logs.loc[1:,"loss"], lw=2, label='training loss')
plt.plot(logs.loc[1:,"val_loss"], lw=2, label='validation loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(logs.loc[1:,"accuracy"], lw=2, label='training accuracy')
plt.plot(logs.loc[1:,"val_accuracy"], lw=2, label='validation accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc='lower right')
plt.show()

In [ ]:
folder_path = "/kaggle/input/private-u-tad-dogs-vs-cats-2024/test/test/"

### Create predictions for all test images

In [ ]:
%%time

folder_path = "/kaggle/input/private-u-tad-dogs-vs-cats-2024/test/test/"

discrimination_threshold = 0.5  # below 0.5 is a cat, above 0.5 is a dog
predictions_dict = {}

for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)

    # save the image name
    file_name = img.split('/')[-1]
    file_no_extension = file_name.split('.')[0]

    img = keras.utils.load_img(img, target_size=image_size)
    img_array = keras.utils.img_to_array(img)
    img_array = keras.ops.expand_dims(img_array, 0)
    predictions = model_new.predict(img_array, verbose=None)
    score = predictions[0][1]
    label = (score > discrimination_threshold )*1

    # save the predictions to a dictionary
    predictions_dict[int(file_no_extension)] = label

### Save predictions to a competition submission file

In [ ]:
submission = pd.DataFrame(predictions_dict.items(), columns=["id", "label"]).sort_values(by='id', ascending=True)
submission.to_csv('submission.csv',index=False)

# print numbers of each class label
submission["label"].value_counts()

In [ ]:
submission